# s3 데이터셋 적재를 위한 테스트 데이터 필터링
- 기간: 220601 ~ 220607

In [1]:
import numpy as np
import pandas as pd
import os
import shutil
import copy
from tqdm import tqdm

import warnings
warnings.filterwarnings(action='ignore')
pd.set_option("display.max_columns", None)

## 1. 데이터셋 필터링

In [2]:
# 현재 작업 디렉토리 가져오기
original_directory = os.getcwd()
print(f"원래 디렉토리: {original_directory}")

원래 디렉토리: c:\Users\aryij\Documents\DataStudy\epoch\final-project


In [3]:
def filter_by_date(dir1="aihub", dir2="2022", start_date="2022-06-01", end_date="2022-06-06"):
    
    # 기본 경로 설정 (data\aihub\2022)
    base_path = os.path.join("data", dir1, dir2, "total_combined")

    folder_list = os.listdir(base_path)

    region_dfs = {}
    region_gps_dfs = {}

    # gps_data, region_data 폴더 내부를 for loop으로 들어간다
    for folder in folder_list:
        folder_path = os.path.join(base_path, folder)
        dataset_list = os.listdir(folder_path)

        for dataset in dataset_list:
            file_path = os.path.join(folder_path, dataset)

            if not dataset.endswith('.csv'):
                print(f"파일 스킵: {dataset} (CSV 파일 아님)")
                continue

            try:
                # 파일 읽기 시도
                df = pd.read_csv(file_path)

                # 파일에 데이터가 없는 경우 스킵
                if df.empty:
                    print(f"파일 스킵: {dataset} (데이터 없음)")
                    continue

                # 데이터프레임에서 "YMD"로 끝나는 컬럼 찾는다 (END_DATE_YMD는 제외)
                ymd_columns = [col for col in df.columns if col.endswith('YMD') and "END" not in col]

                if not ymd_columns:
                    print(f"YMD 컬럼 없음: {dataset}")
                    continue

                print(f"- 데이터셋: {dataset}\n- YMD 컬럼: {ymd_columns}\n")

                # YMD 컬럼 처리
                for ymd_col in ymd_columns:
                    df[ymd_col] = pd.to_datetime(df[ymd_col], errors="coerce")

                # 날짜 필터링 (ymd_columns 중 첫 번째 컬럼)
                filtered_df = df[(df[ymd_columns[0]] >= start_date) & (df[ymd_columns[0]] <= end_date)]

                df_name = dataset.replace(".csv", "")

                if folder == "gps_data":
                    region_gps_dfs[df_name] = filtered_df
                else:
                    region_dfs[df_name] = filtered_df

            except pd.errors.EmptyDataError:
                print(f"파일 스킵: {dataset} (비어 있는 파일)")
            except Exception as e:
                print(f"파일 처리 오류: {dataset} - {e}")

        print(f"===================== {folder} 내부 데이터 필터링 완료 ({start_date} ~ {end_date}) =====================")

    # print(f"===================== 데이터 필터링 완료 (시작 날짜 {start_date}) =====================")
    return region_dfs, region_gps_dfs


In [4]:
# 2022년 데이터셋 combine
filtered_dfs_22, filtered_gps_dfs_22 = filter_by_date("aihub", "2022", "2022-06-01")

- 데이터셋: tn_gps_coord_a_a000007.csv
- YMD 컬럼: ['DT_YMD']

- 데이터셋: tn_gps_coord_a_a000011.csv
- YMD 컬럼: ['DT_YMD']

- 데이터셋: tn_gps_coord_a_a000012.csv
- YMD 컬럼: ['DT_YMD']

- 데이터셋: tn_gps_coord_a_a000013.csv
- YMD 컬럼: ['DT_YMD']

- 데이터셋: tn_gps_coord_a_a000016.csv
- YMD 컬럼: ['DT_YMD']

- 데이터셋: tn_gps_coord_a_a000018.csv
- YMD 컬럼: ['DT_YMD']

- 데이터셋: tn_gps_coord_a_a000019.csv
- YMD 컬럼: ['DT_YMD']

- 데이터셋: tn_gps_coord_a_a000021.csv
- YMD 컬럼: ['DT_YMD']

- 데이터셋: tn_gps_coord_a_a000024.csv
- YMD 컬럼: ['DT_YMD']

- 데이터셋: tn_gps_coord_a_a000028.csv
- YMD 컬럼: ['DT_YMD']

- 데이터셋: tn_gps_coord_a_a000029.csv
- YMD 컬럼: ['DT_YMD']

- 데이터셋: tn_gps_coord_a_a000030.csv
- YMD 컬럼: ['DT_YMD']

- 데이터셋: tn_gps_coord_a_a000034.csv
- YMD 컬럼: ['DT_YMD']

- 데이터셋: tn_gps_coord_a_a000035.csv
- YMD 컬럼: ['DT_YMD']

- 데이터셋: tn_gps_coord_a_a000038.csv
- YMD 컬럼: ['DT_YMD']

- 데이터셋: tn_gps_coord_a_a000041.csv
- YMD 컬럼: ['DT_YMD']

- 데이터셋: tn_gps_coord_a_a000047.csv
- YMD 컬럼: ['DT_YMD']

- 데이터셋: tn_gps_coord_a_a000050.

In [5]:
# # 2023년 데이터셋 combine
# filtered_dfs_23, filtered_gps_dfs_23 = filter_by_date("aihub", "2023", "2023-06-01")

In [6]:
filtered_dfs_22["tn_mvmn_consume_his_이동수단소비내역"]

,TRAVEL_ID,MVMN_SE,PAYMENT_SE,PAYMENT_SEQ,MVMN_SE_NM,PAYMENT_NUM,BRNO,STORE_NM,PAYMENT_DT,PAYMENT_MTHD_SE,PAYMENT_AMT_WON,PAYMENT_ETC,region,PAYMENT_DT_YMD,PAYMENT_DT_MIN
6174,a_a001090,9,교통비,11,항공기,3,NaN,제주항공,2022-06-02 16:23:00,1.0,152500.0,비행기표,capital,2022-06-02,16:23:00
25406,d_d000278,9,교통비,1,항공기,6,1.018195e+09,(주)티웨이항공,2022-06-03 23:22:00,1.0,444950.0,비행기표 할인석,jeju,2022-06-03,23:22:00
26188,d_d000463,9,교통비,2,항공기,2,NaN,NaN,2022-06-05 00:00:00,1.0,137200.0,비행기표,jeju,2022-06-05,00:00:00
27438,d_d000278,9,교통비,2,항공기,6,1.018195e+09,(주)티웨이항공,2022-06-01 18:32:00,1.0,339950.0,비행기표,jeju,2022-06-01,18:32:00
33890,d_d001411,2,렌탈비,1,렌터카(승용/승합/버스 등등),6,2.068690e+09,해피네트웍스,2022-06-04 17:50:00,1.0,621000.0,렌트비,jeju,2022-06-04,17:50:00
35225,d_d000296,2,렌탈비,1,렌터카(승용/승합/버스 등등),4,1.058776e+09,주식회사캐플릭스,2022-06-04 18:08:00,1.0,309200.0,렌트비,jeju,2022-06-04,18:08:00
35231,d_d000014,9,교통비,1,항공기,5,2.208156e+09,(주)케이지이니시스,2022-06-05 10:00:00,1.0,1290400.0,비행기표,jeju,2022-06-05,10:00:00
35600,d_d000518,9,교통비,1,항공기,2,NaN,NaN,2022-06-06 00:00:00,1.0,137600.0,비행기표,jeju,2022-06-06,00:00:00
37736,d_d000432,9,교통비,1,항공기,2,1.018195e+09,(주)티웨이항공,2022-06-02 15:28:00,1.0,136800.0,비행기표,jeju,2022-06-02,15:28:00


In [7]:
# 8월부터 데이터 들어가있어서 필터링 안됨
filtered_dfs_22["tn_move_his_이동내역"]

,TRAVEL_ID,TRIP_ID,START_VISIT_AREA_ID,END_VISIT_AREA_ID,MVMN_CD_1,MVMN_CD_2,region,START_DT_YMD,START_DT_MIN,END_DT_YMD,END_DT_MIN


In [8]:
# 8월부터 데이터 들어가있어서 필터링 안됨
filtered_gps_dfs_22["tn_gps_coord_a_a000007"]

,MOBILE_NUM_ID,X_COORD,Y_COORD,TRAVEL_ID,region,DT_YMD,DT_MIN


## 2. 데이터셋 저장

In [9]:
# 지역별 데이터 통합, 저장 함수 생성
def store_filtered_data(data_dict, dir1="aihub", dir2="2022", is_gps="no", start_date="2022-06-01", end_date="2022-06-06"):
    # 기본 경로 설정 (data\aihub\2022)
    base_path = os.path.join("data", dir1, dir2)

    # gps 데이터가 아니라면
    if is_gps == "no":
        combine_path = os.path.join(base_path, "filtered", "region_data")
    else:
        combine_path = os.path.join(base_path, "filtered", "gps_data")

    # 기존 폴더가 있다면 삭제
    # if os.path.exists(combine_path):
    #     shutil.rmtree(combine_path)

    # filtered 폴더 생성 (2022\filtered)
    os.makedirs(combine_path, exist_ok=True)

    for key in data_dict.keys():
        print(f"{key} 데이터셋 저장 시작 \n저장 경로: {combine_path}\n")
        data_dict[key].to_csv(os.path.join(combine_path, f"{key}.csv"), index=False)
    
    print(f"=================== {dir2} 필터링 데이터 {len(data_dict.keys())}개 저장 완료 ({start_date} ~ {end_date}) ===================")
    print(f"=================== {combine_path} 폴더 내 데이터 갯수: {len(os.listdir(combine_path))} ====================")

In [10]:
store_filtered_data(filtered_dfs_22, is_gps="no")

tn_activity_consume_his_활동소비내역 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\region_data

tn_adv_consume_his_사전소비내역 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\region_data

tn_lodge_consume_his_숙박소비내역 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\region_data

tn_move_his_이동내역 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\region_data

tn_mvmn_consume_his_이동수단소비내역 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\region_data

tn_tour_photo_관광사진 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\region_data

tn_traveller_master_여행객 Master 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\region_data

tn_travel_여행 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\region_data

tn_visit_area_info_방문지정보 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\region_data

=================== 2022 필터링 데이터 9개 저장 완료 (2022-06-01 ~ 2022-06-06) ===================
=================== data\aihub\2022\filtered\region_data 폴더 내 데이터 갯수: 9 ====================


In [11]:
store_filtered_data(filtered_gps_dfs_22, is_gps="yes")

tn_gps_coord_a_a000007 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\gps_data

tn_gps_coord_a_a000011 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\gps_data

tn_gps_coord_a_a000012 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\gps_data

tn_gps_coord_a_a000013 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\gps_data

tn_gps_coord_a_a000016 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\gps_data

tn_gps_coord_a_a000018 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\gps_data

tn_gps_coord_a_a000019 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\gps_data

tn_gps_coord_a_a000021 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\gps_data

tn_gps_coord_a_a000024 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\gps_data

tn_gps_coord_a_a000028 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\gps_data

tn_gps_coord_a_a000029 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\gps_data

tn_gps_coord_a_a000030 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\gps_data

tn_gps_coord_a_a000034 데이터셋 저장 시작 
저장 경로: data\aihub\2022\filtered\gps_data
